In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from IPython.display import display, HTML
from ipywidgets import interact, FloatSlider, IntSlider, Button, Output 
display(HTML("<style>.rendered_html.text_cell_render {max-width:600px; }</style>")) 

# Data utils

Resourses and useful links:

 - ...

In [ ]:
[] == []

In [ ]:
#|default_exp data
#|export

import torch
import inspect
import wrapt
from torch import Tensor as Ten

@wrapt.decorator
def patch_getitem(getitem, instance, args, kwargs):
    """
    Decorator that wraps the `__getitem__` method of its subclasses, and
    applies transforms stored in `instance.fs`.
    """
    i, *_ = args
    x = getitem(i)
    for f in instance.fs: 
        if not isinstance(x, tuple): x = (x,)
        x = f(*x)
    return x


class TransformsMixin(object):
    """
    Mixin that wraps the `__getitem__` method of its subclasses, and
    applies transforms stored in `self.fs`.
    """
    def __init__(self, fs=[]):
        super().__init__()
        self.fs = fs
        self._patch()
        
            
    def _patch(self):
        # Since we are replacing the class method (not the instance's method)
        # we have to make sure we only do it once, and not everytime a sub class
        # of the mixing is initialized.
        ATTR = "_TransformsMixin__getitem"
        if not hasattr(self.__class__, ATTR):
            setattr(self.__class__, ATTR, self.__class__.__getitem__)
            self.__class__.__getitem__ = patch_getitem(self.__class__.__getitem__)

    def add_transform(self, f):
        self.fs.append(f)
        return self
    
    def __rshift__(self, f): return self.add_transform(f)
    
    
def transforms_deco(C_or_c:"Class or instance"):
    """
    Takes a class with a list of transforms stored in `self.fs`
    and wraps its `__getitem__` method, and
    applies transforms stored in `self.fs`.
    """
    if not inspect.isclass(C_or_c): C = C_or_c.__class__
    else: C = C_or_c
    
    ATTR = "_TransformsMixin__getitem"
    if not hasattr(C, ATTR):
        setattr(C, ATTR, C.__getitem__)
        C.__getitem__    = patch_getitem(C.__getitem__)
        C.add_transform  = TransformsMixin.add_transform
        C.__rshift__     = TransformsMixin.__rshift__
    
        
    return C

In [ ]:
"""This is a test cell"""
def f(w, x, y=0, z=1):
    pass

inspect.getfullargspec(f), inspect.signature(f).parameters

In [ ]:
#|export
class Transform():
    def __init__(self, f):
        super().__init__()
        self.f = f
        
    def __call__(self, *args, **kwargs): return self.f(*args, **kwargs)

In [ ]:
"""This is a test cell"""
@transforms_deco
class Testset():
    def __init__(self, n=100, fs=[]):
        super().__init__()
        self.fs = []
        self.data = torch.arange(0,n)
        
        
    def len(self): return len(self.data)
    def __call__(self): print(super().fs)
    def __getitem__(self, i): return self.data[i]


ds  =  Testset(10, fs=[lambda x:x/10, lambda x:x/20])
ds_ = (Testset(10) 
        >> (lambda x:x/10) 
        >> (lambda x:x/20));

print(ds [1])
print(ds_[1])
inspect.signature(ds.__class__.__init__)

In [ ]:
"""This is a test cell"""
class Testset(TransformsMixin):
    def __init__(self, n=100, fs = []):
        super().__init__()
        self.fs = fs
        self.data = torch.arange(0,n)
        
    def len(self): return len(self.data)
    def __call__(self): print(super().fs)
    def __getitem__(self, i): return self.data[i]


ds  =  Testset(10, fs=[lambda x:x/10, lambda x:x/20])
ds_ = (Testset(10) 
        >> (lambda x:x/10) 
        >> (lambda x:x/20));

print(ds [1])
print(ds_[1])